In [ ]:
pip install ortools

In [ ]:
import pandas as pd
import random as rand
from math import floor

#packages for optimisation functions
import scipy.special as ss
from ortools.linear_solver import pywraplp

# package to build interactive charts
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Input Parameters

## Step 1: Assign Input Parameters for further functions.

In [ ]:
#Event 1: Player Bet Probability
# based on players behaviour, can be estimated on historical data
p_bet_draw = 0.05
p_bet_house = 0.62
p_bet_away = 1 - p_bet_draw - p_bet_house
#Event 2: Game Resolution Probability
# the following three can be estimated based on public data
p_draw_win = 0.33
p_house_win = 0.35
p_away_win = 1 - p_draw_win - p_house_win

# Games Qty per 1 week
game_count = 10
# Entry Fee
entry_fee = 20
#Payout per 1 Game Resolution
pay_draw = 25
pay_house = 20
pay_away = 15
# extra payouts for players based on amount of games win from 0 to 10
extra_payout = [0, 0, 0, 0, 0, 0, 20, 80, 160, 10000, 50000]

# Excpected Value Calculation

### Function to Caclulate EV of player

In [ ]:
def excpected_value(entry_fee,game_count,
                    p_draw_win, p_away_win,p_house_win,
                    p_bet_draw, p_bet_away,p_bet_house,
                    pay_draw, pay_away,pay_house, extra_payout):
  # Excpected Payouts is the average expected gross gain of player
  expected_payouts = -entry_fee

  # Probability, that player wins single Game
  probability_win_single = p_draw_win*p_bet_draw + p_away_win*p_bet_away + p_house_win*p_bet_house

  # Expected Player Value from Player Side per Game level
  expected_value_single_win = p_draw_win*pay_draw + p_away_win*pay_away + p_house_win*pay_house

  for n in range(0, game_count + 1):
    # probability to win n out of Games Qty
    prob_win_n_games = probability_win_single**n;
    prob_loose_remainng = (1.0-probability_win_single)**(game_count-n);
    probability_to_win_n_games = prob_win_n_games*prob_loose_remainng*ss.comb(game_count, n)

    # Expected Payouts are Extra Payouts per N correct guesses and Game Payouts * count of guessed games
    expected_payout=n*expected_value_single_win+extra_payout[n]
    expected_payouts += expected_payout * probability_to_win_n_games

  net_gain_player_total = expected_payouts - entry_fee
  return expected_payouts #print('Players Net Gain: ' "{:.2f}".format(net_gain_player_total))

In [ ]:
# run the function with input parameters
excpected_value(entry_fee,game_count,
                    p_draw_win, p_away_win,p_house_win,
                    p_bet_draw, p_bet_away,p_bet_house,
                    pay_draw, pay_away,pay_house, extra_payout)

56.109878675462184

In [ ]:
# function to build chart
def excpected_value_chart(entry_fee,game_count,
                    p_draw_win, p_away_win,p_house_win,
                    p_bet_draw, p_bet_away,p_bet_house,
                    pay_draw, pay_away,pay_house, extra_payout):
  # starts with Negative Entry Fee, because player put their money to play the game
  expected_payouts = -entry_fee
  probability_to_win_n_games_list = []
  payouts = []

  # Probability, that player wins single Game
  probability_win_single = p_draw_win*p_bet_draw + p_away_win*p_bet_away + p_house_win*p_bet_house

  # Expected Player Value from Player Side per Game level
  expected_value_single_win = p_draw_win*pay_draw + p_away_win*pay_away + p_house_win*pay_house

  for n in range(0, game_count + 1):
    # probability to win n out of Games Qty
    prob_win_n_games = probability_win_single**n;
    prob_loose_remainng = (1.0-probability_win_single)**(game_count-n);
    probability_to_win_n_games = prob_win_n_games*prob_loose_remainng*ss.comb(game_count, n)
    probability_to_win_n_games_list.append(probability_to_win_n_games*100)


    # Expected Value for all Games per Player equals to Expected Received Value from Player Side per Game level + Extra Payouts, based on the amount of Games wins
    expected_payout=n*expected_value_single_win+extra_payout[n]

    expected_payouts += expected_payout * probability_to_win_n_games
    payouts.append(expected_payouts)

  correct_guesses = [0,1,2,3,4,5,6,7,8,9,10]

  d = {
        'Correct Guesses, #': correct_guesses,
        'Prob to Guess, %': probability_to_win_n_games_list,
        'Expected Value, $': payouts
     }

  df = pd.DataFrame.from_dict(d, orient = 'index').transpose()

  df['Prob to Guess, %'] = df['Prob to Guess, %'].astype(int)
  df['Expected Value, $'] = df['Expected Value, $'].astype(int)
  df['Correct Guesses, #'] = df['Correct Guesses, #'].astype(int).astype(str)

  fig = px.line(
    df,
    x="Correct Guesses, #",
    y="Prob to Guess, %",
    title='Probability of Correct Guessed Games out of 10 weekly games',
    text= 'Prob to Guess, %'
    )
  fig.update_traces(textposition="top center")
  fig.update_layout(height = 450, width = 650)

  fig2 = px.bar(
    df,
    x="Correct Guesses, #",
    y="Expected Value, $",
    title='EV of Correct Guessed Games out of 10 weekly games',
    text= 'Expected Value, $'
    )
  fig2.update_traces(textposition="outside")
  fig2.update_layout(height = 450, width = 650)
  output = [fig.show(), fig2.show()]

  return fig.show(), fig2.show()

In [ ]:
excpected_value_chart(entry_fee,game_count,
                    p_draw_win, p_away_win,p_house_win,
                    p_bet_draw, p_bet_away,p_bet_house,
                    pay_draw, pay_away,pay_house, extra_payout)

(None, None)

### Minimization

 ## Minimization Function

In [ ]:
# with expected value per game
def minimize_func (entry_fee):
  solver = pywraplp.Solver.CreateSolver("GLOP")

# define payouts at scale = 100
  pay_draw = solver.NumVar(0, solver.infinity(), 'pay_draw')
  pay_away = solver.NumVar(0, solver.infinity(), 'pay_away')
  pay_house = solver.NumVar(0, solver.infinity(), 'pay_house')

  expected_payouts = -entry_fee
  probability_win_single = p_draw_win*p_bet_draw + p_away_win*p_bet_away + p_house_win*p_bet_house
  expected_value_single_win = p_draw_win*pay_draw + p_away_win*pay_away + p_house_win*pay_house

  for n in range(0, game_count +1):
    prob_win_n_games = probability_win_single**n;
    prob_loose_remainng = (1.0-probability_win_single)**(game_count-n);
    probability_to_win_n_games = prob_win_n_games*prob_loose_remainng*ss.comb(game_count, n)
    expected_payout=n*expected_value_single_win+extra_payout[n]
    expected_payouts += expected_payout * probability_to_win_n_games

    # constraints
  solver.Add(pay_draw >= pay_away + 2)
  solver.Add(pay_draw - pay_away <=  3)
  solver.Add(pay_away >= pay_house + 2)
  solver.Add(pay_away - pay_house <= 3)
  solver.Add(pay_house >= 3)
  solver.Add(expected_payouts >= -entry_fee*0.2) #optimize on Casino return is 20% of Entry Fee
  solver.Minimize(expected_payouts)
  status = solver.Solve()
  pay_draw = int(pay_draw.solution_value())
  pay_away = int(pay_away.solution_value())
  pay_house = int(pay_house.solution_value())

  game_payouts = [pay_draw, pay_away, pay_house]

  return game_payouts

In [ ]:
minimize_func (entry_fee = 15)

[7, 5, 3]

In [ ]:
excpected_value(15,game_count,
                    p_draw_win, p_away_win,p_house_win,
                    p_bet_draw, p_bet_away,p_bet_house,
                    7, 5,3, extra_payout)
# for casino it is -9, so even with optimized Game Payouts, casino losses money

9.939688675462202

In [ ]:
#let's iterate entry fee, to find, what is the level, when casino starts to earn
games_payouts = []
entry_fees  = range(15,51)
expected_payouts=[]
pdr = []
pa = []
ph = []
for entry_fee in entry_fees:
  games_payouts = minimize_func (entry_fee)
  pdr.append(games_payouts[0])
  pa.append(games_payouts[1])
  ph.append(games_payouts[2])
  for gp in games_payouts:
    payout_draw = games_payouts[0]
    payout_away = games_payouts[1]
    payout_home = games_payouts[2]
    ep = excpected_value(entry_fee,game_count,
                    p_draw_win, p_away_win,p_house_win,
                    p_bet_draw, p_bet_away,p_bet_house,
                    payout_draw, payout_away,payout_home, extra_payout)
  expected_payouts.append(ep)
d = {
        'Entry Fee': entry_fees,
        'Expected Payouts': expected_payouts,
        'Payout Draw' : pdr,
        'Payout Away' : pa,
        'Payout House' : ph,
     }
df = pd.DataFrame.from_dict(d, orient = 'index').transpose()

In [ ]:
df['Casino Net Win'] = -df['Expected Payouts']
df['Margin %'] = 100*(df['Casino Net Win'] / df['Entry Fee']).astype(float)
X = df['Entry Fee']
y1 = df['Expected Payouts']
y2 = df['Payout Draw']
y3 = df['Payout Away']
y4 = df['Payout House']
y5 = df['Margin %']

# Create traces
fig = go.Figure()
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=X, y=y1,
                    mode='lines+markers',
                    name='Expected Payouts'),
              secondary_y=False)
fig.add_trace(go.Scatter(x=X, y=y2,
                    mode='lines',
                    name='Payout Draw'),
              secondary_y=False)
fig.add_trace(go.Scatter(x=X, y=y3,
                    mode='lines',
                    name='Payout Away'),
              secondary_y=False)
fig.add_trace(go.Scatter(x=X, y=y4,
                    mode='lines',
                    name='Payout House'),
              secondary_y=False)
fig.add_trace(
    go.Scatter(x=X, y=y5, name="Margin %"),
    secondary_y=True,
)
fig.update_layout(height = 600, width = 1000)
# Add figure title
fig.update_layout(
    title_text="Entry Fee and Expected Value optimized on Margin"
)

# Set x-axis title
fig.update_xaxes(title_text="Entry Fee")

# Set y-axes titles
fig.update_yaxes(title_text="<b>$</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>%</b>", secondary_y=True)
fig.show()

Casino is becoming profitable at Entry Fee is 25$.

20% Return is accumulating at Entry Fee is between 31 and 32$.

After Entry Fee crossing 33% the Game Payouts could be increased.

In [ ]:
print('Expected Value per Entry Fee 32$:', int(excpected_value(32,game_count,
                    p_draw_win, p_away_win,p_house_win,
                    p_bet_draw, p_bet_away,p_bet_house,
                    7, 5,3, extra_payout)))

Expected Value per Entry Fee 32$: -7


In [ ]:
print('Casino Return is: ', int(-100*excpected_value(32,game_count,
                    p_draw_win, p_away_win,p_house_win,
                    p_bet_draw, p_bet_away,p_bet_house,
                    7, 5,3, extra_payout) / 32), "%")

Casino Return is:  22 %


## Monte Carlo

In [ ]:
# Simulation of Casino Returns with Entry Fee = 32
entry_fee = 32

def pick_team_type(teams):
  teamIndex = teams.index(rand.choice(teams))
  teamType = teams[teamIndex][0]
  teams[teamIndex][1] -= 1
  if teams[teamIndex][1] == 0:
    del teams[teamIndex]
  return teamType


casino_win = []
margin = []
power = {'strong': 2, 'normal': 1, 'weak': 0}
payout = {
    'draw': minimize_func (entry_fee)[0],
    'away_won': minimize_func (entry_fee)[1],
    'house_won': minimize_func (entry_fee)[2]
}

simulationCount = 1000

playerCount = 1

simulationSum = 0
outcome_sum = 0
outcome_sum_list = []

for simulation in range(simulationCount):
  games = []
  teams = [['normal', 22], ['strong', 4], ['weak', 12]]
  for i in range(10):
    games.append([pick_team_type(teams), pick_team_type(teams)])

  totalMoneyMade = entry_fee * playerCount


  playerCorrectGuesses = [0] * playerCount

  for game in games:
    gp_bet_draw = p_bet_draw
    gp_bet_house = p_bet_house
    gp_draw_win = p_draw_win
    gp_house_win = p_house_win

    r_outcome = rand.random()
    if r_outcome <= gp_house_win:
      outcome = 'house_won'
    elif r_outcome <= gp_house_win + gp_draw_win:
      outcome = 'draw'
    else:
      outcome = 'away_won'

    for player in range(playerCount):
      r_bet = rand.random()
      if r_bet < gp_bet_house:
        bet = 'house_won'
      elif r_bet < gp_bet_draw + gp_bet_house:
        bet = 'draw'
      else:
        bet = 'away_won'

      if bet == outcome:
        playerCorrectGuesses[player] += 1
        totalMoneyMade -= payout[outcome]
        outcome_sum_list.append(payout[outcome])
        outcome_sum += payout[outcome]


  for guesses in playerCorrectGuesses:
    totalMoneyMade -= extra_payout[guesses]

  casino_win.append(totalMoneyMade)

  simulationSum += totalMoneyMade
  margin.append(100*(totalMoneyMade)/ (entry_fee * playerCount))
entry_fee * playerCount
print('Casino Revenue $: ', entry_fee * playerCount)
print('Casino Wins $: ', int(simulationSum / simulationCount))
print('Margin %: ', int(100*(simulationSum / simulationCount) / (entry_fee * playerCount)))

Casino Revenue $:  32
Casino Wins $:  16
Margin %:  50


In [ ]:
import plotly.express as px

fig = px.box(
    # y = range(0,1000),
    x=margin,
    title='Probability of Correct Guessed Games out of 10 weekly games'
    )
fig.update_layout(height = 450, width = 650)
fig.update_traces(orientation='h')
fig.show()